#### !!!!! This tutorial was created by Plot.ly, you will need to install plotly and SQLite to use this notebook.
#### !!!!! It is not included in Anaconda. !!!!!
Check https://plot.ly/python/getting-started/ for installation instructions for Plotly 
and https://www.sqlite.org/download.html for instructions on how to install SQLite.


******

## A Large Data Workflow with Pandas 


##### Data Analysis of 8.2 Million Rows with Python and SQLite

This notebook explores a 3.9Gb CSV file containing NYC's 311 complaints since 2003. It's the most popular data set in [NYC's open data portal](https://nycopendata.socrata.com/data).

This notebook is a primer on out-of-memory data analysis with
- [pandas](http://pandas.pydata.org/): A library with easy-to-use data structures and data analysis tools. Also, interfaces to out-of-memory databases like SQLite.
- [IPython notebook](ipython.org/notebook.html): An interface for writing and sharing python code, text, and plots.
- [SQLite](https://www.sqlite.org/): An self-contained, server-less database that's easy to set-up and query from Pandas.
- [Plotly](https://plot.ly/python/): A platform for publishing beautiful, interactive graphs from Python to the web.

The dataset is too large to load into a Pandas dataframe. So, instead we'll perform out-of-memory aggregations with SQLite and load the result directly into a dataframe with Panda's `iotools`. It's pretty easy to stream a CSV into SQLite and SQLite requires no setup. The SQL query language is pretty intuitive coming from a Pandas mindset.

In [2]:
import pandas as pd
from sqlalchemy import create_engine # database connection
import datetime as dt
from IPython.display import display

import plotly.plotly as py # interactive graphing
from plotly.graph_objs import Bar, Scatter, Marker, Layout 

#### Import the CSV data into SQLite

1. Load the CSV, chunk-by-chunk, into a DataFrame
2. Process the data a bit, strip out uninteresting columns
3. Append it to the SQLite database

In [3]:
display(pd.read_csv('/home/denton/Downloads/2013_311_Service_Requests.csv', nrows=2).head())
display(pd.read_csv('/home/denton/Downloads/2013_311_Service_Requests.csv', nrows=2).tail())

,Unique Key,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Location Type,Incident Zip,Incident Address,...,Bridge Highway Name,Bridge Highway Direction,Road Ramp,Bridge Highway Segment,Garage Lot Name,Ferry Direction,Ferry Terminal Name,Latitude,Longitude,Location
0,25723045,06/11/2013 04:59:00 PM,06/13/2013 12:00:00 PM,DSNY,A - Canine Task Force Citywide,Dirty Conditions,E8 Canine Violation,Sidewalk,11419,94-50 118 STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.692930,-73.828438,"(40.69292971410803, -73.8284375240509)"
1,25723044,06/11/2013 09:35:00 PM,06/18/2013 12:00:00 PM,DSNY,A - Brooklyn,Dirty Conditions,E5 Loose Rubbish,Sidewalk,11215,209 7 AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.670417,-73.978623,"(40.67041672164416, -73.97862322511882)"


,Unique Key,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Location Type,Incident Zip,Incident Address,...,Bridge Highway Name,Bridge Highway Direction,Road Ramp,Bridge Highway Segment,Garage Lot Name,Ferry Direction,Ferry Terminal Name,Latitude,Longitude,Location
0,25723045,06/11/2013 04:59:00 PM,06/13/2013 12:00:00 PM,DSNY,A - Canine Task Force Citywide,Dirty Conditions,E8 Canine Violation,Sidewalk,11419,94-50 118 STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.692930,-73.828438,"(40.69292971410803, -73.8284375240509)"
1,25723044,06/11/2013 09:35:00 PM,06/18/2013 12:00:00 PM,DSNY,A - Brooklyn,Dirty Conditions,E5 Loose Rubbish,Sidewalk,11215,209 7 AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.670417,-73.978623,"(40.67041672164416, -73.97862322511882)"


In [4]:
!wc -l < /home/denton/Downloads/2013_311_Service_Requests.csv # Number of lines in dataset

1611456


In [5]:
disk_engine = create_engine('sqlite:///311_8M.db') # Initializes database with filename 311_8M.db in current directory

In [6]:
# start = dt.datetime.now()
# chunksize = 20000
# j = 0
# index_start = 1

# for df in pd.read_csv('/home/denton/Downloads/2013_311_Service_Requests.csv', chunksize=chunksize, iterator=True, encoding='utf-8'):
    
#     df = df.rename(columns={c: c.replace(' ', '') for c in df.columns}) # Remove spaces from columns

#     df['CreatedDate'] = pd.to_datetime(df['CreatedDate']) # Convert to datetimes
#     df['ClosedDate'] = pd.to_datetime(df['ClosedDate'])

#     df.index += index_start

#     # Remove the un-interesting columns
#     columns = ['Agency', 'CreatedDate', 'ClosedDate', 'ComplaintType', 'Descriptor',
#                'CreatedDate', 'ClosedDate', 'TimeToCompletion',
#                'City']

#     for c in df.columns:
#         if c not in columns:
#             df = df.drop(c, axis=1)    

    
#     j+=1
#     print '{} seconds: completed {} rows'.format((dt.datetime.now() - start).seconds, j*chunksize)

#     df.to_sql('data', disk_engine, if_exists='append')
#     index_start = df.index[-1] + 1

/home/denton/anaconda2/lib/python2.7/site-packages/ipykernel/ipkernel.py:199: DtypeWarning:

Columns (17,39,40) have mixed types. Specify dtype option on import or set low_memory=False.



4 seconds: completed 20000 rows


/home/denton/anaconda2/lib/python2.7/site-packages/ipykernel/ipkernel.py:199: DtypeWarning:

Columns (42,43,44,45) have mixed types. Specify dtype option on import or set low_memory=False.



10 seconds: completed 40000 rows
15 seconds: completed 60000 rows


/home/denton/anaconda2/lib/python2.7/site-packages/ipykernel/ipkernel.py:199: DtypeWarning:

Columns (17,42,43,44,45) have mixed types. Specify dtype option on import or set low_memory=False.



...

394 seconds: completed 1500000 rows
399 seconds: completed 1520000 rows


/home/denton/anaconda2/lib/python2.7/site-packages/ipykernel/ipkernel.py:199: DtypeWarning:

Columns (20,37) have mixed types. Specify dtype option on import or set low_memory=False.



405 seconds: completed 1540000 rows
410 seconds: completed 1560000 rows
415 seconds: completed 1580000 rows


/home/denton/anaconda2/lib/python2.7/site-packages/ipykernel/ipkernel.py:199: DtypeWarning:

Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.



420 seconds: completed 1600000 rows
423 seconds: completed 1620000 rows


###### Preview the table

In [7]:
df = pd.read_sql_query('SELECT * FROM data LIMIT 3', disk_engine)
df.head()

,index,CreatedDate,ClosedDate,Agency,ComplaintType,Descriptor,City
0,1,2013-06-11 16:59:00.000000,2013-06-13 12:00:00.000000,DSNY,Dirty Conditions,E8 Canine Violation,South Richmond Hill
1,2,2013-06-11 21:35:00.000000,2013-06-18 12:00:00.000000,DSNY,Dirty Conditions,E5 Loose Rubbish,BROOKLYN
2,3,2013-06-11 09:12:44.000000,2013-06-11 09:13:00.000000,HRA,Benefit Card Replacement,Food Stamp,None


###### Select just a couple of columns

In [8]:
df = pd.read_sql_query('SELECT Agency, Descriptor FROM data LIMIT 3', disk_engine)
df.head()

,Agency,Descriptor
0,DSNY,E8 Canine Violation
1,DSNY,E5 Loose Rubbish
2,HRA,Food Stamp


###### `LIMIT` the number of rows that are retrieved

In [9]:
df = pd.read_sql_query('SELECT ComplaintType, Descriptor, Agency '
                       'FROM data '
                       'LIMIT 10', disk_engine)
df

,ComplaintType,Descriptor,Agency
0,Dirty Conditions,E8 Canine Violation,DSNY
1,Dirty Conditions,E5 Loose Rubbish,DSNY
2,Benefit Card Replacement,Food Stamp,HRA
3,Missed Collection (All Materials),1 Missed Collection,DSNY
4,Sanitation Condition,15 Street Cond/Dump-Out/Drop-Off,DSNY
5,Sanitation Condition,12 Dead Animals,DSNY
6,Sanitation Condition,12 Dead Animals,DSNY
7,Sanitation Condition,15 Street Cond/Dump-Out/Drop-Off,DSNY
8,Root/Sewer/Sidewalk Condition,Trees and Sidewalks Program,DPR
9,Dirty Conditions,E12 Illegal Dumping Surveillance,DSNY


###### Filter rows with `WHERE`

In [10]:
df = pd.read_sql_query('SELECT ComplaintType, Descriptor, Agency '
                       'FROM data '
                       'WHERE Agency = "NYPD" '
                       'LIMIT 10', disk_engine)
df.head()

,ComplaintType,Descriptor,Agency
0,Illegal Parking,Blocked Sidewalk,NYPD
1,Noise - Street/Sidewalk,Loud Music/Party,NYPD
2,Noise - Street/Sidewalk,Loud Talking,NYPD
3,Noise - Commercial,Loud Music/Party,NYPD
4,Noise - Street/Sidewalk,Loud Talking,NYPD


###### Filter multiple values in a column with `WHERE` and `IN`

In [11]:
df = pd.read_sql_query('SELECT ComplaintType, Descriptor, Agency '
                       'FROM data '
                       'WHERE Agency IN ("NYPD", "DOB")'
                       'LIMIT 10', disk_engine)
df.head()

,ComplaintType,Descriptor,Agency
0,Plumbing,Plumbing-Defective/Leaking/Not Maintained,DOB
1,Plumbing,Plumbing-Defective/Leaking/Not Maintained,DOB
2,Plumbing,Plumbing-Defective/Leaking/Not Maintained,DOB
3,Elevator,Elevator - Defective/Not Working,DOB
4,Investigations and Discipline (IAD),Plumbing Work - Unlicensed/Illegal/Improper Wo...,DOB


###### Find the unique values in a column with `DISTINCT`

In [12]:
df = pd.read_sql_query('SELECT DISTINCT City FROM data', disk_engine)
df.head()

,City
0,South Richmond Hill
1,BROOKLYN
2,None
3,STATEN ISLAND
4,NEW YORK


###### Query value counts with `COUNT(*)` and `GROUP BY`

In [13]:
df = pd.read_sql_query('SELECT Agency, COUNT(*) as `num_complaints`'
                       'FROM data '
                       'GROUP BY Agency ', disk_engine)

df.head()

,Agency,num_complaints
0,3-1-1,996
1,ACS,1
2,CAU,2
3,CCRB,4
4,CHALL,1042


###### Order the results with `ORDER` and `-`
Housing and Development Dept receives the most complaints

In [16]:
tls.set_credentials_file(username='ddeloss', api_key='0uwykwidtt')

In [17]:
df = pd.read_sql_query('SELECT Agency, COUNT(*) as `num_complaints`'
                       'FROM data '
                       'GROUP BY Agency '
                       'ORDER BY -num_complaints', disk_engine)

py.iplot([Bar(x=df.Agency, y=df.num_complaints)], filename='311/most common complaints by agency')

###### Heat / Hot Water is the most common complaint

In [18]:
df = pd.read_sql_query('SELECT ComplaintType, COUNT(*) as `num_complaints`, Agency '
                       'FROM data '
                       'GROUP BY `ComplaintType` '
                       'ORDER BY -num_complaints', disk_engine)


most_common_complaints = df # used later
py.iplot({
    'data': [Bar(x=df['ComplaintType'], y=df.num_complaints)],
    'layout': { 
        'margin': {'b': 150}, # Make the bottom margin a bit bigger to handle the long text
        'xaxis': {'tickangle': 40}} # Angle the labels a bit
    }, filename='311/most common complaints by complaint type')

*This graph is interactive. Click-and-drag horizontally to zoom, shift-click to pan, double click to autoscale*

##### What's the most common complaint in each city?

First, let's see how many cities are recorded in the dataset

In [19]:
len(pd.read_sql_query('SELECT DISTINCT City FROM data', disk_engine))

581

Yikes - let's just plot the 10 most complained about cities

In [20]:
df = pd.read_sql_query('SELECT City, COUNT(*) as `num_complaints` '
                                'FROM data '
                                'GROUP BY `City` '
                       'ORDER BY -num_complaints '
                       'LIMIT 10 ', disk_engine)
df

,City,num_complaints
0,BROOKLYN,462724
1,NEW YORK,305377
2,BRONX,279323
3,None,169750
4,STATEN ISLAND,73555
5,Jamaica,25139
6,Flushing,15919
7,Astoria,13023
8,FLUSHING,11047
9,JAMAICA,10829


Flushing and FLUSHING, Jamaica and JAMAICA... the complaints are case sensitive.

###### Perform case insensitive queries with `GROUP BY` with `COLLATE NOCASE`

In [21]:
df = pd.read_sql_query('SELECT City, COUNT(*) as `num_complaints` '
                        'FROM data '
                        'GROUP BY `City` '
                       'COLLATE NOCASE '
                       'ORDER BY -num_complaints '
                       'LIMIT 11 ', disk_engine)
df

,City,num_complaints
0,BROOKLYN,462724
1,NEW YORK,305377
2,BRONX,279323
3,None,169750
4,STATEN ISLAND,73555
5,JAMAICA,35968
6,FLUSHING,26966
7,ASTORIA,22003
8,RIDGEWOOD,15917
9,FAR ROCKAWAY,11030


In [22]:
cities = list(df.City)
cities.remove(None)

In [23]:
traces = [] # the series in the graph - one trace for each city

for city in cities:
    df = pd.read_sql_query('SELECT ComplaintType, COUNT(*) as `num_complaints` '
                           'FROM data '
                           'WHERE City = "{}" COLLATE NOCASE '
                           'GROUP BY `ComplaintType` '
                           'ORDER BY -num_complaints'.format(city), disk_engine)

    traces.append(Bar(x=df['ComplaintType'], y=df.num_complaints, name=city.capitalize()))

In [24]:
py.iplot({'data': traces, 'layout': Layout(barmode='stack', xaxis={'tickangle': 40}, margin={'b': 150})}, filename='311/complaints by city stacked')

*You can also `click` on the legend entries to hide/show the traces. Click-and-drag to zoom in and shift-drag to pan.*

Now let's normalize these counts. This is super easy now that this data has been reduced into a dataframe.

In [25]:
for trace in traces:
    trace['y'] = 100.*trace['y']/sum(trace['y'])

In [26]:
py.iplot({'data': traces, 
          'layout': Layout(
                barmode='group',
                xaxis={'tickangle': 40, 'autorange': False, 'range': [-0.5, 16]},
                yaxis={'title': 'Percent of Complaints by City'},
                margin={'b': 150},
                title='Relative Number of 311 Complaints by City')
         }, filename='311/relative complaints by city', validate=False)

- New York is loud
- Staten Island is moldy, wet, and vacant
- Flushing's muni meters are broken 
- Trash collection is great in the Bronx
- Woodside doesn't like its graffiti

Click and drag to pan across the graph and see more of the complaints. 

### Part 2: SQLite time series with Pandas

######  Filter SQLite rows with timestamp strings: `YYYY-MM-DD hh:mm:ss`

In [36]:
df = pd.read_sql_query('SELECT ComplaintType, CreatedDate, City '
                       'FROM data '
                       'WHERE CreatedDate < "2013-11-16 23:47:00" '
                       'AND CreatedDate > "2013-11-16 23:45:00"', disk_engine)

df

,ComplaintType,CreatedDate,City
0,Noise - Street/Sidewalk,2013-11-16 23:46:17.000000,NEW YORK
1,Illegal Parking,2013-11-16 23:45:19.000000,WOODHAVEN


######  Pull out the hour unit from timestamps with `strftime`


In [37]:
df = pd.read_sql_query('SELECT CreatedDate, '
                              'strftime(\'%H\', CreatedDate) as hour, '
                              'ComplaintType '
                       'FROM data '
                       'LIMIT 5 ', disk_engine)
df.head()

,CreatedDate,hour,ComplaintType
0,2013-06-11 16:59:00.000000,16,Dirty Conditions
1,2013-06-11 21:35:00.000000,21,Dirty Conditions
2,2013-06-11 09:12:44.000000,09,Benefit Card Replacement
3,2013-11-14 14:38:00.000000,14,Missed Collection (All Materials)
4,2013-11-14 13:55:00.000000,13,Sanitation Condition


######  Count the number of complaints (rows) per hour with `strftime`, `GROUP BY`, and `count(*)`

In [38]:
df = pd.read_sql_query('SELECT CreatedDate, '
                               'strftime(\'%H\', CreatedDate) as hour,  '
                               'count(*) as `Complaints per Hour`'
                       'FROM data '
                       'GROUP BY hour', disk_engine)

df.head()

,CreatedDate,hour,Complaints per Hour
0,2013-09-02 00:21:45.000000,00,620808
1,2013-06-03 01:22:41.000000,01,14581
2,2013-02-24 02:50:04.000000,02,9947
3,2013-12-12 03:29:10.000000,03,6504
4,2013-01-31 04:37:11.000000,04,5501


In [39]:
py.iplot({
    'data': [Bar(x=df['hour'], y=df['Complaints per Hour'])],
    'layout': Layout(xaxis={'title': 'Hour in Day'},
                     yaxis={'title': 'Number of Complaints'})}, filename='311/complaints per hour')

######  Filter noise complaints by hour

In [40]:
df = pd.read_sql_query('SELECT CreatedDate, '
                               'strftime(\'%H\', CreatedDate) as `hour`,  '
                               'count(*) as `Complaints per Hour`'
                       'FROM data '
                       'WHERE ComplaintType IN ("Noise", '
                                               '"Noise - Street/Sidewalk", '
                                               '"Noise - Commercial", '
                                               '"Noise - Vehicle", '
                                               '"Noise - Park", '
                                               '"Noise - House of Worship", '
                                               '"Noise - Helicopter", '
                                               '"Collection Truck Noise") '
                       'GROUP BY hour', disk_engine)

display(df.head(n=2))

py.iplot({
    'data': [Bar(x=df['hour'], y=df['Complaints per Hour'])],
    'layout': Layout(xaxis={'title': 'Hour in Day'},
                     yaxis={'title': 'Number of Complaints'},
                     title='Number of Noise Complaints in NYC by Hour in Day'
                    )}, filename='311/noise complaints per hour')

,CreatedDate,hour,Complaints per Hour
0,2013-05-28 00:20:00.000000,00,9446
1,2013-05-12 01:39:00.000000,01,7645


######  Segregate complaints by hour

In [41]:
complaint_traces = {} # Each series in the graph will represent a complaint
complaint_traces['Other'] = {}

for hour in range(1, 24):
    hour_str = '0'+str(hour) if hour < 10 else str(hour)
    df = pd.read_sql_query('SELECT  CreatedDate, '
                                   'ComplaintType ,'
                                   'strftime(\'%H\', CreatedDate) as `hour`,  '
                                   'COUNT(*) as num_complaints '
                           'FROM data '
                           'WHERE hour = "{}" '
                           'GROUP BY ComplaintType '
                           'ORDER BY -num_complaints'.format(hour_str), disk_engine)
    
    complaint_traces['Other'][hour] = sum(df.num_complaints)
    
    # Grab the 7 most common complaints for that hour
    for i in range(7):
        complaint = df.get_value(i, 'ComplaintType')
        count = df.get_value(i, 'num_complaints')
        complaint_traces['Other'][hour] -= count
        if complaint in complaint_traces:
            complaint_traces[complaint][hour] = count
        else:
            complaint_traces[complaint] = {hour: count}

In [42]:
traces = []
for complaint in complaint_traces:
    traces.append({
        'x': range(25),
        'y': [complaint_traces[complaint].get(i, None) for i in range(25)],
        'name': complaint,
        'type': 'bar'
    })

py.iplot({
    'data': traces, 
    'layout': {
        'barmode': 'stack',
        'xaxis': {'title': 'Hour in Day'},
        'yaxis': {'title': 'Number of Complaints'},
        'title': 'The 7 Most Common 311 Complaints by Hour in a Day'
    }}, filename='311/most common complaints by hour')

##### Aggregated time series

First, create a new column with timestamps rounded to the previous 15 minute interval

In [43]:
minutes = 15
seconds = 15*60

df = pd.read_sql_query('SELECT CreatedDate, '
                               'datetime(('
                                   'strftime(\'%s\', CreatedDate) / {seconds}) * {seconds}, \'unixepoch\') interval '
                       'FROM data '
                       'LIMIT 10 '.format(seconds=seconds), disk_engine)

display(df.head())

,CreatedDate,interval
0,2013-06-11 16:59:00.000000,2013-06-11 16:45:00
1,2013-06-11 21:35:00.000000,2013-06-11 21:30:00
2,2013-06-11 09:12:44.000000,2013-06-11 09:00:00
3,2013-11-14 14:38:00.000000,2013-11-14 14:30:00
4,2013-11-14 13:55:00.000000,2013-11-14 13:45:00


Then, `GROUP BY` that interval and `COUNT(*)`

In [44]:
minutes = 15
seconds = minutes*60

df = pd.read_sql_query('SELECT datetime(('
                                   'strftime(\'%s\', CreatedDate) / {seconds}) * {seconds}, \'unixepoch\') interval ,'
                               'COUNT(*) as "Complaints / interval"'
                       'FROM data '
                       'GROUP BY interval '
                       'ORDER BY interval '
                       'LIMIT 500'.format(seconds=seconds), disk_engine)

display(df.head())
display(df.tail())

,interval,Complaints / interval
0,2013-01-01 00:00:00,1668
1,2013-01-01 00:15:00,1
2,2013-01-01 00:30:00,2
3,2013-01-01 00:45:00,4
4,2013-01-01 01:00:00,5


,interval,Complaints / interval
495,2013-01-06 04:45:00,3
496,2013-01-06 05:15:00,1
497,2013-01-06 05:45:00,3
498,2013-01-06 06:15:00,1
499,2013-01-06 06:30:00,2


In [45]:
py.iplot(
    {
        'data': [{
            'x': df.interval,
            'y': df['Complaints / interval'],
            'type': 'bar'
        }],
        'layout': {
            'title': 'Number of 311 Complaints per 15 Minutes'
        }
}, filename='311/complaints per 15 minutes')

In [46]:
hours = 24
minutes = hours*60
seconds = minutes*60

df = pd.read_sql_query('SELECT datetime(('
                                   'strftime(\'%s\', CreatedDate) / {seconds}) * {seconds}, \'unixepoch\') interval ,'
                               'COUNT(*) as "Complaints / interval"'
                       'FROM data '
                       'GROUP BY interval '
                       'ORDER BY interval'.format(seconds=seconds), disk_engine)

In [47]:
py.iplot(
    {
        'data': [{
            'x': df.interval,
            'y': df['Complaints / interval'],
            'type': 'bar'
        }],
        'layout': {
            'title': 'Number of 311 Complaints per Day'
        }
}, filename='311/complaints per day')

### Learn more

- Find more open data sets on [Data.gov](https://data.gov) and [NYC Open Data](https://nycopendata.socrata.com)
- Learn how to setup [MySql with Pandas and Plotly](http://moderndata.plot.ly/graph-data-from-mysql-database-in-python/)
- Add [interactive widgets to IPython notebooks](http://moderndata.plot.ly/widgets-in-ipython-notebook-and-plotly/) for customized data exploration
- Big data workflows with [HDF5 and Pandas](http://stackoverflow.com/questions/14262433/large-data-work-flows-using-pandas)
- [Interactive graphing with Plotly](https://plot.ly/python/)

In [48]:
from IPython.core.display import HTML
import urllib2
HTML(urllib2.urlopen('https://raw.githubusercontent.com/plotly/python-user-guide/css-updates/custom.css').read())